In [1]:
import pandas as pd
import numpy as np
import glob as glob
import os

In [18]:
prokka_path = '../../results/annotations/'

# Load metadata
metadata = pd.read_csv('../../results/opa_metadata_locus.csv', index_col = 0)

# Checking which opa genes found by our script were not found by Prokka
filenames = glob.glob(prokka_path + '*.csv')
filenames = np.sort(filenames)
merged = pd.DataFrame()
prokka_all = pd.DataFrame()
for filename in filenames:
    strain = os.path.basename(filename)[:-4]
    metadata_strain = metadata[metadata['strain']==strain]
    metadata_strain.sort_values('stop_term', inplace = True, ignore_index = True)
    
    prokka = pd.read_csv(filename, index_col = 0)
    prokka['strain'] = strain
    prokka_all = pd.concat([prokka_all, prokka])
    prokka_all.reset_index(drop = True, inplace = True)
    
    metadata_strain_plus = metadata_strain[metadata_strain['strand']==1]
    metadata_strain_minus = metadata_strain[metadata_strain['strand']==-1]

    merged = pd.concat([merged, pd.merge_asof(metadata_strain_plus, prokka, left_on = 'stop_term', right_on = 'end', by = ['strain', 'strand'], direction = 'nearest', tolerance = 700, suffixes = ('_metadata', '_prokka'))])
    merged = pd.concat([merged, pd.merge_asof(metadata_strain_minus, prokka, left_on = 'stop_term', right_on = 'start', by = ['strain', 'strand'], direction = 'nearest', tolerance = 700, suffixes = ('_metadata', '_prokka'))])
    
    merged = merged.sort_values(by = ['strain', 'stop_term'])
    merged.reset_index(drop = True, inplace = True)

'Number of opa genes found by our script that were not found by Prokka: ' + str(len(merged[merged['locus_tag'].isnull()]))

/tmp/ipykernel_366049/905981934.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_strain.sort_values('stop_term', inplace = True, ignore_index = True)
/tmp/ipykernel_366049/905981934.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_strain.sort_values('stop_term', inplace = True, ignore_index = True)
/tmp/ipykernel_366049/905981934.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_strain.sort_values(

'Number of opa genes found by our script that were not found by Prokka: 2'

In [19]:
np.unique(merged['gene'].dropna())

array(['piiC_1', 'piiC_10', 'piiC_11', 'piiC_12', 'piiC_13', 'piiC_14',
       'piiC_15', 'piiC_2', 'piiC_3', 'piiC_4', 'piiC_5', 'piiC_6',
       'piiC_7', 'piiC_8', 'piiC_9'], dtype=object)

In [20]:
# All of the opa genes identified by our script are named piiC_X

In [21]:
# Checking which piiC genes found by Prokka were not found by our script
prokka_all_nonan = prokka_all.dropna(subset = ['gene'])
prokka_piiC = prokka_all_nonan[prokka_all_nonan['gene'].str.contains('piiC')]

metadata_plus = metadata[metadata['strand']==1]
metadata_minus = metadata[metadata['strand']==-1]
prokka_piiC_plus = prokka_piiC[prokka_piiC['strand']==1]
prokka_piiC_minus = prokka_piiC[prokka_piiC['strand']==-1]

piiC_plus = pd.merge_asof(prokka_piiC_plus.sort_values(by = 'end'), metadata_plus.sort_values(by = 'stop_term'), left_on = 'end', right_on = 'stop_term', by = ['strand', 'strain'], direction = 'nearest', tolerance = 700, suffixes = ('_prokka', '_metadata'))
piiC_minus = pd.merge_asof(prokka_piiC_minus.sort_values(by = 'start'), metadata_minus.sort_values(by = 'stop_term'), left_on = 'start', right_on = 'stop_term', by = ['strand', 'strain'], direction = 'nearest', tolerance = 700, suffixes = ('_prokka', '_metadata'))
piiC = pd.concat([piiC_plus, piiC_minus])

piiC[piiC['stop_term'].isnull()].sort_values('strain')

,chromosome_prokka,gene,locus_tag,source,start_prokka,end,strand,strain,chromosome_metadata,start_cr,...,ref_lcb_stop,ref_lcb_strand,query_lcb_start,query_lcb_stop,query_lcb_strand,length_ratio,start_reordered,start_reordered_flipped,strand_flipped,locus
100,gnl|X|10231_1,piiC_1,10231_00060,prokka,69280.0,69814.0,-1,10231,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,gnl|X|10268_1,piiC_1,10268_00060,prokka,68739.0,69273.0,-1,10268,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,gnl|X|10272_1,piiC_1,10272_00060,prokka,69020.0,69554.0,-1,10272,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,gnl|X|10328_1,piiC_1,10328_00060,prokka,69016.0,69550.0,-1,10328,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,gnl|X|10500_1,piiC_1,10500_00061,prokka,68986.0,69523.0,-1,10500,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,gnl|X|10524_1,piiC_1,10524_00059,prokka,69000.0,69534.0,-1,10524,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,gnl|X|10529_1,piiC_1,10529_00063,prokka,68892.0,69426.0,-1,10529,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,gnl|X|10531_1,piiC_1,10531_00061,prokka,68992.0,69526.0,-1,10531,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,gnl|X|10562_1,piiC_1,10562_00060,prokka,68421.0,68955.0,-1,10562,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,gnl|X|10574_1,piiC_1,10574_00060,prokka,68722.0,69256.0,-1,10574,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Interestingly, all of the piiC found by Prokka are the same ones as found by BLAST! Exactly overlapping!